In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Importing Necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

Loading the dataset

In [ ]:
cct1 = pd.read_csv (r'D:\pyt\Consumer_Complaints_train.csv')
cct2 = pd.read_csv (r'D:\pyt\Consumer_Complaints_test_share.csv')

In [ ]:
cct1.shape , cct2.shape

((478421, 18), (119606, 17))

Checking Event rate-

In [ ]:
cct1['Consumer disputed?'].value_counts()

No     376990
Yes    101431
Name: Consumer disputed?, dtype: int64

In [ ]:
376990/478421, 101431/478421

(0.7879879854772261, 0.21201201452277388)

Labelling data

In [ ]:
cct1['data']='train'
cct2['data']='test'

Merging the data

In [ ]:
cc_all=pd.concat([cct1,cct2],axis=0)

Data Preparation

In [ ]:
cc_all.dtypes

Date received                   object
Product                         object
Sub-product                     object
Issue                           object
Sub-issue                       object
Consumer complaint narrative    object
Company public response         object
Company                         object
State                           object
ZIP code                        object
Tags                            object
Consumer consent provided?      object
Submitted via                   object
Date sent to company            object
Company response to consumer    object
Timely response?                object
Consumer disputed?              object
Complaint ID                     int64
data                            object
dtype: object

In [ ]:
# Converting columns into date time format
for col in ['Date received','Date sent to company']:
    cc_all[col] = pd.to_datetime(cc_all[col], infer_datetime_format=True)

In [ ]:
# Time required to forword the dispute to company
cc_all['Day_Diff'] = pd.to_numeric(cc_all['Date sent to company']-cc_all['Date received'])
cc_all.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,data,Day_Diff
0,2014-05-15,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,48342,Older American,NaN,Web,2014-05-16,Closed with explanation,Yes,No,856103,train,86400000000000
1,2014-09-18,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,18042,NaN,NaN,Referral,2014-09-24,Closed,Yes,No,1034666,train,518400000000000
2,2014-03-13,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,92427,NaN,NaN,Referral,2014-04-03,Closed with non-monetary relief,Yes,No,756363,train,1814400000000000
3,2015-07-17,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,305XX,Older American,Consent provided,Web,2015-07-17,Closed with monetary relief,Yes,No,1474177,train,0
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,02127,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,1132572,train,691200000000000


In [ ]:
# Dropping unnecessary features
cc_all=cc_all.drop(['Date received','Date sent to company'], axis=1)

In [ ]:
cc_all.head()
cc_all.dtypes

Product                         object
Sub-product                     object
Issue                           object
Sub-issue                       object
Consumer complaint narrative    object
Company public response         object
Company                         object
State                           object
ZIP code                        object
Tags                            object
Consumer consent provided?      object
Submitted via                   object
Company response to consumer    object
Timely response?                object
Consumer disputed?              object
Complaint ID                     int64
data                            object
Day_Diff                         int64
dtype: object

In [ ]:
#  Checking unique values in all object type features
for col in cc_all.select_dtypes(['object']).columns:
    print(col,'-',cc_all[col].nunique())

Product - 12
Sub-product - 47
Issue - 95
Sub-issue - 68
Consumer complaint narrative - 92118
Company public response - 10
Company - 3437
State - 62
ZIP code - 27342
Tags - 3
Consumer consent provided? - 4
Submitted via - 6
Company response to consumer - 7
Timely response? - 2
Consumer disputed? - 2
data - 2


In [ ]:
# Checking Null values
cc_all.isnull().sum()

Product                              0
Sub-product                     173225
Issue                                0
Sub-issue                       365685
Consumer complaint narrative    504376
Company public response         484859
Company                              0
State                             4764
ZIP code                          4774
Tags                            513950
Consumer consent provided?      428676
Submitted via                        1
Company response to consumer         0
Timely response?                     0
Consumer disputed?              119606
Complaint ID                         0
data                                 0
Day_Diff                             0
dtype: int64

Label Encoding

In [ ]:
for col in ['Sub-issue','Sub-product','Consumer complaint narrative','Company public response','Tags','Consumer consent provided?']:
    colname = col.replace('-','_').replace('?','').replace(" ",'_') + '_isNan'
    cc_all[colname] = np.where (pd.isnull(cc_all[col]),1,0)
    cc_all.drop([col],axis=1,inplace=True) 

In [ ]:
cc_all['Timely response?'] = np.where(cc_all['Timely response?']=='Yes',1,0)

In [ ]:
cc_all['Consumer_complaint_narrative_isNan'].head()

0    1
1    1
2    1
3    0
4    1
Name: Consumer_complaint_narrative_isNan, dtype: int32

In [ ]:
for col in cc_all.select_dtypes(['object']).columns:
    print(col,'-',cc_all[col].nunique())

Product - 12
Issue - 95
Company - 3437
State - 62
ZIP code - 27342
Submitted via - 6
Company response to consumer - 7
Consumer disputed? - 2
data - 2


In [ ]:
# Dropping unnecessary features
cc_all.drop(['Company','ZIP code'], axis=1,inplace=True)

In [ ]:
for col in cc_all.select_dtypes(['object']).columns:
    print(col,'-',cc_all[col].nunique())

Product - 12
Issue - 95
State - 62
Submitted via - 6
Company response to consumer - 7
Consumer disputed? - 2
data - 2


In [ ]:
m = cc_all['Issue'].value_counts()
for col in m.axes[0][0:12]:
    colname = 'Issue_' + col.replace(' ','_').replace(',','_')
    cc_all[colname] = np.where(cc_all['Issue'] == col,1,0)
del cc_all['Issue']


In [ ]:
n = cc_all['State'].value_counts()
for col in n.axes[0][0:15]:
    colname ='State_' + col.replace(' ','_').replace(',','_')
    cc_all[colname] = np.where(cc_all['State']==col,1,0)
del cc_all['State']

In [ ]:
# Creating Dummy-
for col in ['Product','Company response to consumer','Submitted via']:
  var=pd.get_dummies(cc_all[col])
  pd.concat([var,cc_all], axis=1)
  cc_all.drop([col], axis=1,inplace=True)

In [ ]:
cc_all.head()

,Timely response?,Consumer disputed?,Complaint ID,data,Day_Diff,Sub_issue_isNan,Sub_product_isNan,Consumer_complaint_narrative_isNan,Company_public_response_isNan,Tags_isNan,...,State_NJ,State_PA,State_IL,State_VA,State_MD,State_OH,State_NC,State_MI,State_AZ,State_WA
0,1,No,856103,train,86400000000000,1,1,1,1,0,...,0,0,0,0,0,0,0,1,0,0
1,1,No,1034666,train,518400000000000,1,0,1,1,1,...,0,1,0,0,0,0,0,0,0,0
2,1,No,756363,train,1814400000000000,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,No,1474177,train,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,No,1132572,train,691200000000000,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
 cc_all['Consumer disputed?'].unique()
 cc_train = cc_all[cc_all['data'] == 'train']
 cc_test = cc_all[cc_all['data'] == 'test']
 cc_train.drop(['data'], axis=1,inplace=True)
 cc_test.drop(['data'], axis=1,inplace=True)
 cc_train['Consumer disputed?'] = np.where(cc_train['Consumer disputed?']=='Yes',1,0)
 cc_train.head()

C:\Users\Yogesh S\AppData\Local\Temp\ipykernel_20380\3667721248.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cc_train.drop(['data'], axis=1,inplace=True)
C:\Users\Yogesh S\AppData\Local\Temp\ipykernel_20380\3667721248.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cc_test.drop(['data'], axis=1,inplace=True)
C:\Users\Yogesh S\AppData\Local\Temp\ipykernel_20380\3667721248.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

,Timely response?,Consumer disputed?,Complaint ID,Day_Diff,Sub_issue_isNan,Sub_product_isNan,Consumer_complaint_narrative_isNan,Company_public_response_isNan,Tags_isNan,Consumer_consent_provided_isNan,...,State_NJ,State_PA,State_IL,State_VA,State_MD,State_OH,State_NC,State_MI,State_AZ,State_WA
0,1,0,856103,86400000000000,1,1,1,1,0,1,...,0,0,0,0,0,0,0,1,0,0
1,1,0,1034666,518400000000000,1,0,1,1,1,1,...,0,1,0,0,0,0,0,0,0,0
2,1,0,756363,1814400000000000,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,1474177,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,1132572,691200000000000,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Creating input features and target feature for train data
X = cc_train.drop(['Consumer disputed?','Complaint ID'],axis=1)
y = cc_train ['Consumer disputed?']

In [ ]:
# Balancing the data-
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[(0, 376990), (1, 376990)]


In [ ]:
y_resampled.value_counts()

0    376990
1    376990
Name: Consumer disputed?, dtype: int64

In [ ]:
#Splitting the data--
from sklearn.model_selection import train_test_split
X_train,  X_test, y_train, y_test = train_test_split (X_resampled, y_resampled, test_size=0.3, random_state=1)

In [ ]:
# Creating input features and target feature for test data
test_x = cc_test.drop(['Consumer disputed?','Complaint ID'],axis=1)
test_y = cc_test ['Consumer disputed?']


Model Building with logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logr=LogisticRegression()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
parameter = {'penalty':['l2'],'C':[1,2,3],'max_iter':[100,200,300]}
classifier_reg = RandomizedSearchCV(logr,parameter,n_jobs=-1,cv=5,error_score='raise')

In [ ]:
#Fitting to the model
classifier_reg.fit(X_train,y_train)

C:\Users\Yogesh S\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5, error_score='raise', estimator=LogisticRegression(),
                   n_jobs=-1,
                   param_distributions={'C': [1, 2, 3],
                                        'max_iter': [100, 200, 300],
                                        'penalty': ['l2']})

In [ ]:
print(classifier_reg.best_params_)

{'penalty': 'l2', 'max_iter': 100, 'C': 1}


In [ ]:
print(classifier_reg.best_score_)

0.5274675729353399


In [ ]:
y_pred = classifier_reg.predict(X_test)
y_pred2 = classifier_reg.predict_proba(X_test)

Validation Technique

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,roc_auc_score

In [ ]:
score = accuracy_score (y_pred,y_test)
print(score)

0.5277195681583066


In [ ]:
class_repo=classification_report(y_pred,y_test)
print(class_repo)

              precision    recall  f1-score   support

           0       0.68      0.52      0.59    148363
           1       0.37      0.54      0.44     77831

    accuracy                           0.53    226194
   macro avg       0.53      0.53      0.52    226194
weighted avg       0.58      0.53      0.54    226194



In [ ]:
roc_auc=roc_auc_score(y_test,y_pred)
print(roc_auc)

0.5280546864905155


Actual test prediction

In [ ]:
y1_pred2_proba = classifier_reg.predict_proba(test_x)
y1_pred2 = classifier_reg.predict(test_x)

In [ ]:
pd.DataFrame(list(zip(cc_test['Complaint ID'],list(y1_pred2))),columns=['Complaint ID','Consumer disputed?']).to_csv('consumer_services_project_log_reg.csv')

*Next* Model-Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
parameter = {'n_estimators':[100,200,300], 'max_depth':[5,10,15], 'min_samples_split':[2,4,8] }
rfc_classifier = RandomizedSearchCV(rfc,parameter,scoring='accuracy',cv=5,error_score='raise',n_jobs=-1)

In [ ]:
rfc_classifier.fit(X_train,y_train)

RandomizedSearchCV(cv=5, error_score='raise',
                   estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'max_depth': [5, 10, 15],
                                        'min_samples_split': [2, 4, 8],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy')

In [ ]:
# Pickling the model- Random forest 
import pickle
pickle.dump(rfc_classifier, open('rfc_classifier.pkl', 'wb'))

In [ ]:
pickled_rf = pickle.load(open('rfc_classifier.pkl', 'rb'))

In [ ]:
print(pickled_rf.best_params_)
print(pickled_rf.best_score_)

{'n_estimators': 300, 'min_samples_split': 2, 'max_depth': 15}
0.5721371938285584


In [ ]:
y_rfc_pred = pickled_rf.predict(X_test)

In [ ]:
print(accuracy_score(y_rfc_pred,y_test))
print(confusion_matrix(y_rfc_pred,y_test))
print(roc_auc_score(y_rfc_pred,y_test))

0.5737199041530722
[[57683 41251]
 [55171 72089]]
0.5747577404743238


In [ ]:
# Predicting actual test data
y_rfc_pred2 = pickled_rf.predict(test_x)

In [ ]:
print(y_rfc_pred2)

[0 0 1 ... 0 0 0]


In [ ]:
pd.DataFrame(list(zip(cc_test['Complaint ID'],list(y_rfc_pred2))),columns=['Complaint ID','Consumer disputed?']).to_csv('consumer_services_project_rf.csv')

Next Model- Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbclass = GradientBoostingClassifier()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
parameter = {'n_estimators':[100,200,300], 'max_depth':[3,5,7,10], 'min_samples_split':[2,4,6,8]}
gbm_classifier = RandomizedSearchCV(gbclass,parameter,cv=5,random_state =30,error_score='raise', n_jobs=-1)

In [ ]:
gbm_classifier.fit(X_train,y_train)

RandomizedSearchCV(cv=5, error_score='raise',
                   estimator=GradientBoostingClassifier(), n_jobs=-1,
                   param_distributions={'max_depth': [3, 5, 7, 10],
                                        'min_samples_split': [2, 4, 6, 8],
                                        'n_estimators': [100, 200, 300]},
                   random_state=30)

In [ ]:
import pickle 
pickle.dump(gbm_classifier, open('gbm_classifier.pkl', 'wb'))

In [ ]:
print (gbm_classifier.best_params_)
print(gbm_classifier.best_score_)

{'n_estimators': 200, 'min_samples_split': 4, 'max_depth': 10}
0.5744127364135536


In [ ]:
y_gb_pred = gbm_classifier.predict(X_test)

In [ ]:
from sklearn.metrics import roc_auc_score,classification_report,confusion_matrix
print(classification_report (y_test,y_gb_pred))
print(confusion_matrix(y_test,y_gb_pred))
print(roc_auc_score(y_test,y_gb_pred))

              precision    recall  f1-score   support

           0       0.59      0.52      0.55    112854
           1       0.57      0.64      0.60    113340

    accuracy                           0.58    226194
   macro avg       0.58      0.58      0.58    226194
weighted avg       0.58      0.58      0.58    226194

[[58331 54523]
 [40864 72476]]
0.5781639292349251


In [ ]:
# Predicting actual test data
y_gb_pred2 = gbm_classifier.predict(test_x)
print(y_gb_pred2)
# Predicting probabilities
y_gb_pred2_proba = gbm_classifier.predict_proba(test_x)
print(y_gb_pred2_proba)

[0 0 1 ... 0 0 0]
[[0.63459433 0.36540567]
 [0.63079106 0.36920894]
 [0.41143312 0.58856688]
 ...
 [0.64501197 0.35498803]
 [0.53281785 0.46718215]
 [0.50133015 0.49866985]]


In [ ]:
pd.DataFrame(list(zip(cc_test['Complaint ID'],list(y_gb_pred2))),columns=['Complaint ID','Consumer disputed?']).to_csv('consumer_services_project_gbm.csv')

In [ ]:
# Model no-3 XGBOOST
from xgboost import XGBClassifier
xgb = XGBClassifier()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
parameter = {'n_estimators':[100,200,300], 'max_depth':[3,5,7,10], 'min_samples_split':[2,4,6,8]}
xgb_clf = RandomizedSearchCV(xgb,parameter,cv=5,random_state =32,error_score='raise', n_jobs=-1)

In [ ]:
xgb_clf.fit(X_train,y_train)

[00:40:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "min_samples_split" } are not used.



RandomizedSearchCV(cv=5, error_score='raise',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraint...
                                           max_cat_to_onehot=None,
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None, ...),
                   n_jobs=-1,
                   param_distributions={'max_depth': [3, 5, 7, 10],
                                        'min_samples_split': [2, 4, 6, 8],
                                        'n_estimators': [100, 200, 300]},
                   random_state=32)

In [ ]:
import pickle
pickle.dump(xgb_clf,open('xgb_clf2','wb'))

In [ ]:
y_pred3 = xgb_clf.predict(X_test)

In [ ]:
from sklearn.metrics import roc_auc_score,classification_report,confusion_matrix
print(classification_report (y_test,y_pred3))
print(confusion_matrix(y_test,y_pred3))
print(roc_auc_score(y_test,y_pred3))

              precision    recall  f1-score   support

           0       0.58      0.51      0.55    112854
           1       0.57      0.63      0.60    113340

    accuracy                           0.57    226194
   macro avg       0.57      0.57      0.57    226194
weighted avg       0.57      0.57      0.57    226194

[[58005 54849]
 [41597 71743]]
0.573485951899531


In [ ]:
# Predicting actual test data
y_pred31 = xgb_clf.predict(test_x)
print(y_pred31)

[0 0 1 ... 0 0 0]


In [ ]:
pd.DataFrame(list(zip(cc_test['Complaint ID'],list(y_pred31))),columns=['Complaint ID','Consumer disputed?']).to_csv('consumer_services_project_xgb.csv')

In [ ]:
### final comment---As we have seen no major changes seen by changing model. 
##So considering gradient boosting and submitting it with score 0.55
#Thank U